# 스케줄러 및 모델 로드

디퓨전 파이프라인은 특정 사용 사례에 맞게 파이프라인을 조정하기 위해 혼합하고 일치시킬 수 있는 상호 교환 가능한 스케줄러 및 모델 모음입니다. 스케줄러는 노이즈 제거 단계 수 및 노이즈 제거된 샘플을 찾는 알고리즘과 같은 전체 노이즈 제거 프로세스를 캡슐화합니다. 스케줄러는 매개변수화되거나 훈련되지 않으므로 메모리를 많이 차지하지 않습니다. 모델은 일반적으로 노이즈가 있는 입력에서 노이즈가 적은 샘플로 이동하는 정방향 패스에만 관련됩니다.

이 가이드에서는 파이프라인을 사용자 지정하기 위해 스케줄러와 모델을 로드하는 방법을 보여줍니다. 이 가이드 전체에서 [stable-diffusion-v1-5/stable-diffusion-v1-5](https://hf.co/stable-diffusion-v1-5/stable-diffusion-v1-5) 체크포인트를 사용하므로 먼저 로드하겠습니다.

In [ ]:
import torch
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16, use_safetensors=True
).to("cuda")

`pipeline.scheduler` 속성을 사용하여 이 파이프라인이 사용하는 스케줄러를 확인할 수 있습니다.

In [ ]:
pipeline.scheduler
PNDMScheduler {
  "_class_name": "PNDMScheduler",
  "_diffusers_version": "0.21.4",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "num_train_timesteps": 1000,
  "set_alpha_to_one": false,
  "skip_prk_steps": true,
  "steps_offset": 1,
  "timestep_spacing": "leading",
  "trained_betas": null
}

## 스케줄러 로드

스케줄러는 다양한 스케줄러에서 사용할 수 있는 구성 파일로 정의됩니다. [SchedulerMixin.from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/schedulers/overview#diffusers.SchedulerMixin.from_pretrained) 메서드를 사용하여 스케줄러를 로드하고 `subfolder` 매개변수를 지정하여 구성 파일을 파이프라인 리포지토리의 올바른 하위 폴더에 로드합니다.

예를 들어 [DDIMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/ddim#diffusers.DDIMScheduler)를 로드하려면 다음을 수행합니다.

In [ ]:
from diffusers import DDIMScheduler, DiffusionPipeline

ddim = DDIMScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")

그런 다음 새로 로드된 스케줄러를 파이프라인에 전달할 수 있습니다.

In [ ]:
pipeline = DiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", scheduler=ddim, torch_dtype=torch.float16, use_safetensors=True
).to("cuda")

## 스케줄러 비교

스케줄러는 고유한 장단점이 있어 파이프라인에 가장 적합한 스케줄러를 정량적으로 비교하기 어렵습니다. 일반적으로 노이즈 제거 속도와 노이즈 제거 품질 간에 절충해야 합니다. 다양한 스케줄러를 시도하여 사용 사례에 가장 적합한 스케줄러를 찾는 것이 좋습니다. `pipeline.scheduler.compatibles` 속성을 호출하여 파이프라인과 호환되는 스케줄러를 확인합니다.

다음 프롬프트와 시드에서 [LMSDiscreteScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lms_discrete#diffusers.LMSDiscreteScheduler), [EulerDiscreteScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/euler#diffusers.EulerDiscreteScheduler), [EulerAncestralDiscreteScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/euler_ancestral#diffusers.EulerAncestralDiscreteScheduler) 및 [DPMSolverMultistepScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/multistep_dpm_solver#diffusers.DPMSolverMultistepScheduler)를 비교해 보겠습니다.

In [ ]:
import torch
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16, use_safetensors=True
).to("cuda")

prompt = "화성에서 말을 타는 우주 비행사의 사진, 고해상도, 고화질."
generator = torch.Generator(device="cuda").manual_seed(8)

파이프라인 스케줄러를 변경하려면 [from_config()](https://huggingface.co/docs/diffusers/main/en/api/configuration#diffusers.ConfigMixin.from_config) 메서드를 사용하여 다른 스케줄러의 `pipeline.scheduler.config`를 파이프라인에 로드합니다.

<hfoptions id="schedulers">
<hfoption id="LMSDiscreteScheduler">

[LMSDiscreteScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lms_discrete#diffusers.LMSDiscreteScheduler)는 일반적으로 기본 스케줄러보다 고품질 이미지를 생성합니다.

In [ ]:
from diffusers import LMSDiscreteScheduler

pipeline.scheduler = LMSDiscreteScheduler.from_config(pipeline.scheduler.config)
image = pipeline(prompt, generator=generator).images[0]
image

</hfoption>
<hfoption id="EulerDiscreteScheduler">

[EulerDiscreteScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/euler#diffusers.EulerDiscreteScheduler)는 단 30단계 만에 고품질 이미지를 생성할 수 있습니다.

In [ ]:
from diffusers import EulerDiscreteScheduler

pipeline.scheduler = EulerDiscreteScheduler.from_config(pipeline.scheduler.config)
image = pipeline(prompt, generator=generator).images[0]
image

</hfoption>
<hfoption id="EulerAncestralDiscreteScheduler">

[EulerAncestralDiscreteScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/euler_ancestral#diffusers.EulerAncestralDiscreteScheduler)는 단 30단계 만에 고품질 이미지를 생성할 수 있습니다.

In [ ]:
from diffusers import EulerAncestralDiscreteScheduler

pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(pipeline.scheduler.config)
image = pipeline(prompt, generator=generator).images[0]
image

</hfoption>
<hfoption id="DPMSolverMultistepScheduler">

[DPMSolverMultistepScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/multistep_dpm_solver#diffusers.DPMSolverMultistepScheduler)는 속도와 품질 간의 균형을 제공하며 단 20단계 만에 고품질 이미지를 생성할 수 있습니다.

In [ ]:
from diffusers import DPMSolverMultistepScheduler

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)
image = pipeline(prompt, generator=generator).images[0]
image

</hfoption>
</hfoptions>

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/patrickvonplaten/images/resolve/main/diffusers_docs/astronaut_lms.png" />
    <figcaption class="mt-2 text-center text-sm text-gray-500">LMSDiscreteScheduler</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/patrickvonplaten/images/resolve/main/diffusers_docs/astronaut_euler_discrete.png" />
    <figcaption class="mt-2 text-center text-sm text-gray-500">EulerDiscreteScheduler</figcaption>
  </div>
</div>
<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/patrickvonplaten/images/resolve/main/diffusers_docs/astronaut_euler_ancestral.png" />
    <figcaption class="mt-2 text-center text-sm text-gray-500">EulerAncestralDiscreteScheduler</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/patrickvonplaten/images/resolve/main/diffusers_docs/astronaut_dpm.png" />
    <figcaption class="mt-2 text-center text-sm text-gray-500">DPMSolverMultistepScheduler</figcaption>
  </div>
</div>

대부분의 이미지는 매우 유사하며 품질 면에서 비슷합니다. 다시 말하지만, 특정 사용 사례에 따라 달라지므로 여러 다른 스케줄러를 실행하고 결과를 비교하는 것이 좋은 방법입니다.

### Flax 스케줄러

Flax 스케줄러를 비교하려면 스케줄러 상태를 모델 매개변수에 추가로 로드해야 합니다. 예를 들어 [FlaxStableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.FlaxStableDiffusionPipeline)의 기본 스케줄러를 매우 빠른 `FlaxDPMSolverMultistepScheduler`를 사용하도록 변경해 보겠습니다.

> [!WARNING]
> `FlaxLMSDiscreteScheduler` 및 `FlaxDDPMScheduler`는 아직 [FlaxStableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.FlaxStableDiffusionPipeline)과 호환되지 않습니다.

In [ ]:
import jax
import numpy as np
from flax.jax_utils import replicate
from flax.training.common_utils import shard
from diffusers import FlaxStableDiffusionPipeline, FlaxDPMSolverMultistepScheduler

scheduler, scheduler_state = FlaxDPMSolverMultistepScheduler.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    subfolder="scheduler"
)
pipeline, params = FlaxStableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    scheduler=scheduler,
    revision="bf16",
    dtype=jax.numpy.bfloat16,
)
params["scheduler"] = scheduler_state

그런 다음 Flax의 TPU 호환성을 활용하여 여러 이미지를 병렬로 생성할 수 있습니다. 사용 가능한 각 장치에 대한 모델 매개변수 복사본을 만들고 원하는 수의 이미지를 생성하기 위해 입력을 분할해야 합니다.

In [ ]:
# 병렬 장치당 이미지 1개 생성(TPUv2-8 또는 TPUv3-8에서 8개)
prompt = "화성에서 말을 타는 우주 비행사의 사진, 고해상도, 고화질."
num_samples = jax.device_count()
prompt_ids = pipeline.prepare_inputs([prompt] * num_samples)

prng_seed = jax.random.PRNGKey(0)
num_inference_steps = 25

# 입력 및 rng 분할
params = replicate(params)
prng_seed = jax.random.split(prng_seed, jax.device_count())
prompt_ids = shard(prompt_ids)

images = pipeline(prompt_ids, params, prng_seed, num_inference_steps, jit=True).images
images = pipeline.numpy_to_pil(np.asarray(images.reshape((num_samples,) + images.shape[-3:])))

## 모델

모델은 [ModelMixin.from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/models/overview#diffusers.ModelMixin.from_pretrained) 메서드에서 로드되며, 이 메서드는 최신 버전의 모델 가중치와 구성을 다운로드하고 캐시합니다. 최신 파일을 로컬 캐시에서 사용할 수 있는 경우 [from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/models/overview#diffusers.ModelMixin.from_pretrained)는 다시 다운로드하는 대신 캐시의 파일을 재사용합니다.

`subfolder` 인수를 사용하여 하위 폴더에서 모델을 로드할 수 있습니다. 예를 들어 [runwayml/stable-diffusion-v1-5](https://hf.co/runwayml/stable-diffusion-v1-5)의 모델 가중치는 [unet](https://hf.co/runwayml/stable-diffusion-v1-5/tree/main/unet) 하위 폴더에 저장됩니다.

In [ ]:
from diffusers import UNet2DConditionModel

unet = UNet2DConditionModel.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="unet", use_safetensors=True)

[리포지토리](https://huggingface.co/google/ddpm-cifar10-32/tree/main)에서 직접 로드할 수도 있습니다.

In [ ]:
from diffusers import UNet2DModel

unet = UNet2DModel.from_pretrained("google/ddpm-cifar10-32", use_safetensors=True)

모델 변형을 로드하고 저장하려면 [ModelMixin.from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/models/overview#diffusers.ModelMixin.from_pretrained) 및 [ModelMixin.save_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/models/overview#diffusers.ModelMixin.save_pretrained)에서 `variant` 인수를 지정합니다.

In [ ]:
from diffusers import UNet2DConditionModel

unet = UNet2DConditionModel.from_pretrained(
    "runwayml/stable-diffusion-v1-5", subfolder="unet", revision="non-ema", use_safetensors=True
)
unet.save_pretrained("./local-unet", variant="non_ema")